In [19]:
import os
import requests
import numpy as np
import pandas as pd
import regex as re
import tensorflow as tf
import pickle
from utils import *
from processor import PreProcessor
from model import DialectClassifier

np.random.seed(3)
tf.random.set_seed(3)
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


saveFileName = "data/dialects.csv"
weightsPath = "weights/full_grams_cbow_100_twitter.mdl"
isDownload = False

In [2]:
dialects_df = readCSV("data/dialect_dataset.csv")
dialects_arr = np.array(dialects_df,dtype="str")

ids = dialects_arr[:,0]
dialects = dialects_arr[:,1]
print(dialects_arr.shape)

(458197, 2)


In [3]:
if isDownload:
    tweets = get_tweets(ids)
    print(tweets.shape)
    
    dialects_data = np.array([ids,tweets,dialects]).T
    saveCSV(dialects_data,saveFileName,True)
else:    
    dialects_data = readCSV(saveFileName,True,"str")
    
np.random.shuffle(dialects_data)
print(dialects_data.shape)

ids,tweets,dialects = dialects_data[:,0],dialects_data[:,1],dialects_data[:,2]
dataSize = dialects_data.shape[0]

(458197, 3)


In [36]:
preprocessor = PreProcessor(w2vPath=weightsPath,isRemove = True)

encodedDialects,no_classes,labels2idx,idx2labels = labelsEncoder(dialects)
oneHotDialects = oneHotEncoder(encodedDialects,no_classes)
print(oneHotDialects.shape)

pickle.dump(idx2labels,open("idx2labels.pkl","wb"))

idx = np.random.randint(0,ids.shape[0])
print(f"id : {ids[idx]}\ntweet : {tweets[idx]}\ndialect : {dialects[idx]}")

Loading Word2vec
(458197, 18)
id : 1125049558495526784
tweet : @numidie10 أنتى ق م ر ى .😍.هادو مغردين وخلاص😏😍😍😂❤
dialect : LY


In [5]:
print(preprocessor.identifyMentions(tweets[idx]))
print(preprocessor.identifyMentions("@imesS0852_boy https:///gomeogme/fmofw @1998 email@gmail.com @ahmedmohamed \n @gn_494irnirhr"))

قبل نقعد بال48 ساعه فايق المهم نرقد في الضي 
توا نرك فيها في عز الصهد والرطوبة وانفيق جوي فوق الريح
 @USER https:///gomeogme/fmofw  @USER email@gmail.com  @USER 
  @USER


In [6]:
print(preprocessor.identifyURL(tweets[idx]))
print(preprocessor.identifyURL("@imesS0852_boy https://www.google.com/in-go/123456:8080-98 @1998 email@gmail.com www.facebook.com @ahmedmohamed \n @gn_494irnirhr www.youtube.com/video=15020"))

قبل نقعد بال48 ساعه فايق المهم نرقد في الضي 
توا نرك فيها في عز الصهد والرطوبة وانفيق جوي فوق الريح
@imesS0852_boy  @URL  @1998 email@gmail.com  @URL  @ahmedmohamed 
 @gn_494irnirhr  @URL 


In [7]:
print(preprocessor.identifyHashTag(tweets[idx]))
print(preprocessor.identifyHashTag("#hash1_2 @imesS0852_boy https://www.google.com/in/123456:8080-98 #hash2_3 @1998 email@gmail.com www.facebook.com @ahmedmohamed \n @gn_494irnirhr #hash4_5"))

قبل نقعد بال48 ساعه فايق المهم نرقد في الضي 
توا نرك فيها في عز الصهد والرطوبة وانفيق جوي فوق الريح
 @HASH @imesS0852_boy https://www.google.com/in/123456:8080-98  @HASH @1998 email@gmail.com www.facebook.com @ahmedmohamed 
 @gn_494irnirhr  @HASH 


In [8]:
print(preprocessor.removePunctuations(tweets[idx]))
print(preprocessor.removePunctuations("'How are you doing ?!',\nsays mohamed."))

قبل نقعد بال48 ساعه فايق المهم نرقد في الضي 
توا نرك فيها في عز الصهد والرطوبة وانفيق جوي فوق الريح
How are you doing 
says mohamed


In [9]:
print(preprocessor.identifyEmojis(tweets[idx]))
print(preprocessor.identifyEmojis(u'This is a smiley face \U0001f602'))

قبل نقعد بال48 ساعه فايق المهم نرقد في الضي 
توا نرك فيها في عز الصهد والرطوبة وانفيق جوي فوق الريح
This is a smiley face  @EMOJI 


In [10]:
print("before cleaning : ", tweets[idx])
print("after cleaning : ",preprocessor.cleanTweet(tweets[idx].lower()))
print(preprocessor.cleanTweet("@mohamedessam98_20  this is a cleaned tweet \U0001f602 ! @AIM ! from wwww.twitter.com. #CLEANTWEET"))

before cleaning :  قبل نقعد بال48 ساعه فايق المهم نرقد في الضي 
توا نرك فيها في عز الصهد والرطوبة وانفيق جوي فوق الريح
after cleaning :  قبل نقعد بال48 ساعه فايق المهم نرقد في الضي توا نرك فيها في عز الصهد والرطوبه وانفيق جوي فوق الريح
@USER this is a cleaned tweet @EMOJI @USER from @URL @HASH


In [11]:
cleanedTweets = preprocessor.cleanTweets(tweets,isRemove=True)
preprocessor.buildVocabulary(cleanedTweets)

In [12]:
tweetsTokens,masks = preprocessor.tokenizeTweets(cleanedTweets)
print(tweetsTokens.shape)

(458197, 70)


In [13]:
preTrainedEmbed = preprocessor.loadPreTrainedEmbeddings()
print(preTrainedEmbed.shape)

(446654, 100)


In [17]:
embed_dim = 100
hidden_dim = 128
voc_len = preprocessor.LEN_VOC+1
max_sen_len = preprocessor.MAX_SEN_LEN
batch_size = 64
epochs = 10
lr = 0.001


X_train,X_test,Y_train,Y_test,masks_train,masks_test = splitData(tweetsTokens,oneHotDialects,masks,splitPercent = 0.7)

optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.CategoricalCrossentropy()



In [18]:
dialectClassifier = DialectClassifier(voc_len,max_sen_len,embed_dim,hidden_dim,no_classes)
dialectClassifier.compile(optimizer = optimizer,loss = loss, metrics = "accuracy")
hist = dialectClassifier.fit(X_train,Y_train,batch_size = batch_size,
                             epochs = epochs,validation_split=0.3)

Epoch 1/10
3509/3509 [==============================] - 185s 51ms/step - loss: 1.8667 - accuracy: 0.4187 - val_loss: 1.6242 - val_accuracy: 0.4949
Epoch 2/10
3509/3509 [==============================] - 177s 50ms/step - loss: 1.0857 - accuracy: 0.6677 - val_loss: 1.6949 - val_accuracy: 0.4917
Epoch 3/10
3509/3509 [==============================] - 177s 50ms/step - loss: 0.6426 - accuracy: 0.8062 - val_loss: 1.9725 - val_accuracy: 0.4769
Epoch 4/10
3509/3509 [==============================] - 178s 51ms/step - loss: 0.4447 - accuracy: 0.8646 - val_loss: 2.2831 - val_accuracy: 0.4686
Epoch 5/10
3509/3509 [==============================] - 177s 50ms/step - loss: 0.3392 - accuracy: 0.8959 - val_loss: 2.5448 - val_accuracy: 0.4653
Epoch 6/10
3509/3509 [==============================] - 177s 51ms/step - loss: 0.2809 - accuracy: 0.9115 - val_loss: 2.7919 - val_accuracy: 0.4596
Epoch 7/10
3509/3509 [==============================] - 178s 51ms/step - loss: 0.2412 - accuracy: 0.9239 - val_loss: 2

In [23]:
dialectClassifier.evaluate(X_test,Y_test)

4296/4296 [==============================] - 26s 6ms/step - loss: 3.3843 - accuracy: 0.4580


[3.3842661380767822, 0.45801687240600586]

In [22]:
dialectClassifier.save('models/dialectClassifierRNN')

INFO:tensorflow:Assets written to: models/dialectClassifierRNN\assets


INFO:tensorflow:Assets written to: models/dialectClassifierRNN\assets
